In [1]:
import sympy
import numpy as np
from tqdm import tqdm
import dill
dill.settings["recurse"] = True
from sympy import Rational as R 

In [2]:
(
    reference_x,
    reference_y,
    physical_x_0,
    physical_y_0,
    physical_x_1,
    physical_y_1,
    physical_x_2,
    physical_y_2,
) = sympy.symbols("x y x_0 y_0 x_1 y_1 x_2 y_2")

f_interpolation = sympy.Matrix(sympy.symbols("f_0:3"))
g_interpolation = sympy.Matrix(sympy.symbols("g_0:3"))
s = sympy.symbols("s")

In [3]:
v0 = {reference_x: 0, reference_y: 0}
v1 = {reference_x: 1, reference_y: 0}
v2 = {reference_x: 0, reference_y: 1}

# Get basis functions

In [4]:
monomial_basis = sympy.Matrix(
[
 1,
 reference_x,
 reference_y,
]
)

In [5]:
Vander = sympy.Matrix(np.zeros((3, 3)))

for i, basis in enumerate(monomial_basis):
    
    Vander[i, 0] = basis.subs(v0)
    Vander[i, 1] = basis.subs(v1)
    Vander[i, 2] = basis.subs(v2)

In [6]:
lagrange_p1_basis = Vander.inv()@monomial_basis

# Get mapping function

In [7]:
mapping_basis = Vander.inv() @ monomial_basis

global_x = (
      mapping_basis[0] * physical_x_0
    + mapping_basis[1] * physical_x_1
    + mapping_basis[2] * physical_x_2
)
global_y = (
      mapping_basis[0] * physical_y_0
    + mapping_basis[1] * physical_y_1
    + mapping_basis[2] * physical_y_2
)

mapping_function = sympy.Matrix([global_x, global_y])

In [8]:
l0 = ((physical_x_2 - physical_x_1) ** 2 + (physical_y_2 - physical_y_1) ** 2) ** R(1, 2)
l1 = ((physical_x_2 - physical_x_0) ** 2 + (physical_y_2 - physical_y_0) ** 2) ** R(1, 2)
l2 = ((physical_x_1 - physical_x_0) ** 2 + (physical_y_1 - physical_y_0) ** 2) ** R(1, 2)

In [9]:
J_cofactor_T = mapping_function.jacobian([reference_x, reference_y]).cofactor_matrix().T
J = J_cofactor_T / J_cofactor_T.det()
J_inv = J.inv()

In [10]:
u = sympy.Matrix([sympy.Function("u")(reference_x, reference_y)])
v = sympy.Matrix([sympy.Function("v")(reference_x, reference_y)])

In [11]:
grad_u = u.jacobian([reference_x, reference_y])@J
grad_v = v.jacobian([reference_x, reference_y])@J

weak_form = grad_u@grad_v.T

In [12]:
N = len(lagrange_p1_basis)

weak_form_functional = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part_neuman_0 = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part_neuman_1 = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part_neuman_2 = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = lagrange_p1_basis[idx]
    second = lagrange_p1_basis[jdx]

    A = weak_form.subs({u[0]:first, v[0]:second})
    A = sympy.simplify(A)
    B = first * second
    B = sympy.simplify(B)

    sym_int_A = sympy.integrate(A, (reference_y, 0, R(1) - reference_x), (reference_x, 0, R(1)))
    sym_int_B = sympy.integrate(B, (reference_y, 0, R(1) - reference_x), (reference_x, 0, R(1)))

    edge_0 = B.subs({reference_x: s, reference_y: 1-s})
    edge_1 = B.subs({reference_x: 0, reference_y: s})
    edge_2 = B.subs({reference_x: s, reference_y: 0})
    
    symp_int_edge_0 = sympy.integrate(edge_0, (s, 0, 1))
    symp_int_edge_1 = sympy.integrate(edge_1, (s, 0, 1))
    symp_int_edge_2 = sympy.integrate(edge_2, (s, 0, 1))
    
    weak_form_functional[idx][jdx] = sym_int_A
    weak_form_right_part[idx][jdx] = sym_int_B
    
    weak_form_right_part_neuman_0[idx][jdx] = symp_int_edge_0 * l0
    weak_form_right_part_neuman_1[idx][jdx] = symp_int_edge_1 * l1
    weak_form_right_part_neuman_2[idx][jdx] = symp_int_edge_2 * l2
        

weak_form_functional = sympy.Matrix(weak_form_functional)
weak_form_right_part = sympy.Matrix(weak_form_right_part)

weak_form_functional = weak_form_functional * abs(J.inv().det())
weak_form_right_part = weak_form_right_part * abs(J.inv().det())

100%|██████████| 9/9 [01:00<00:00,  6.70s/it]


In [13]:
weak_form_functional = sympy.Matrix(weak_form_functional)
weak_form_right_part = sympy.Matrix(weak_form_right_part)
weak_form_right_part_neuman_0 = sympy.Matrix(weak_form_right_part_neuman_0)
weak_form_right_part_neuman_1 = sympy.Matrix(weak_form_right_part_neuman_1)
weak_form_right_part_neuman_2 = sympy.Matrix(weak_form_right_part_neuman_2)

f_interpolation = sympy.Matrix(f_interpolation)
g_interpolation = sympy.Matrix(g_interpolation)

In [14]:
lambdify_symbols = [    
    physical_x_0,
    physical_y_0,
    physical_x_1,
    physical_y_1,
    physical_x_2,
    physical_y_2
]

In [ ]:
weak_form_functional_lambdified = sympy.lambdify(lambdify_symbols, weak_form_functional, cse=True)

In [ ]:
weak_form_right_part_lambdified = sympy.lambdify([*lambdify_symbols, *f_interpolation], weak_form_right_part@f_interpolation, cse=True)

In [15]:
weak_form_right_part_M_lambdified = sympy.lambdify(lambdify_symbols, weak_form_right_part, cse=True)

In [ ]:
weak_form_right_part_neuman_0_lambdified = sympy.lambdify([*lambdify_symbols, *g_interpolation], weak_form_right_part_neuman_0@g_interpolation, cse=True)
weak_form_right_part_neuman_1_lambdified = sympy.lambdify([*lambdify_symbols, *g_interpolation], weak_form_right_part_neuman_1@g_interpolation, cse=True)
weak_form_right_part_neuman_2_lambdified = sympy.lambdify([*lambdify_symbols, *g_interpolation], weak_form_right_part_neuman_2@g_interpolation, cse=True)

In [ ]:
def weak_form_right_part_neuman_lambdified(x_0, y_0, x_1, y_1, x_2, y_2, g_0, g_1, g_2, idx):
    return [weak_form_right_part_neuman_0_lambdified,
            weak_form_right_part_neuman_1_lambdified,
            weak_form_right_part_neuman_2_lambdified,
            ][idx](x_0, y_0, x_1, y_1, x_2, y_2, g_0, g_1, g_2)

In [ ]:
import dill

dill.settings["recurse"] = True
dill.dump(weak_form_functional_lambdified, open("../calculations/lagrange_linear_poisson_matrix", "wb"))
dill.dump(weak_form_right_part_lambdified, open("../calculations/lagrange_linear_poisson_right_values", "wb"))
dill.dump(weak_form_right_part_neuman_lambdified, open("../calculations/lagrange_linear_poisson_right_values_neuman", "wb"))

In [16]:
dill.dump(weak_form_right_part_M_lambdified, open("../calculations/lagrange_linear_poisson_right_M_values", "wb"))